In [ ]:
import os
import ee
import geemap
import math
Map = geemap.Map()
ee.Initialize()
Map

In [ ]:
import urllib.request

In [ ]:
geemap.ee_search()

In [ ]:
def generateGrid(xmin, xmax, dx, y1, y2):
    xx = ee.List.sequence(xmin, ee.Number(xmax).subtract(dx), dx)
    cells = xx.map(func_uke).flatten()
    return ee.FeatureCollection(cells)

def func_uke(x):
    x1 = ee.Number(x)
    x2 = ee.Number(x).add(ee.Number(dx))
    coords = ee.List([x1, y1, x2, y2])
    rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)
    return ee.Feature(rect)

In [ ]:
def func_feh(feature):                 
    arableland = feature.getNumber('arableland')
    forest = feature.getNumber('forest')
    grassland = feature.getNumber('grassland')
    shrubland = feature.getNumber('shrubland')
    wetlands = feature.getNumber('wetlands')
    waterbody = feature.getNumber('waterbody')
    tundra = feature.getNumber('tundra')
    atificalsurface = feature.getNumber('atificalsurface')
    bareground = feature.getNumber('bareground')
    permanentsnow = feature.getNumber('permanentsnow')
    Land = feature.getNumber('Land')   
    return feature.set({
        'arableland':arableland.divide(Land),
        'forest':forest.divide(Land),
        'grassland':grassland.divide(Land),
        'shrubland':shrubland.divide(Land),
        'wetlands':wetlands.divide(Land),
        'waterbody':waterbody.divide(Land),
        'tundra':tundra.divide(Land),
        'atificalsurface':atificalsurface.divide(Land),
        'bareground':bareground.divide(Land),
        'permanentsnow':permanentsnow.divide(Land),
        'Land':Land.divide(Land)
     })

In [ ]:
import requests
from retrying import retry
@retry(stop_max_attempt_number=40, wait_fixed=2000)
def get_request(url):
    response = urllib.request.urlretrieve(url, filename=out_ndvi_stats)
    return response

In [ ]:
dx = 1/120
xmin = 0
xmax = 180
ymin = -58
ymax = 90
yrange = round((ymax - ymin  - 0.0001)/dx)+1
print(yrange)

In [ ]:
lat_list = ['n_80_n','n_75_n','n_70_n','n_65_n','n_60_n','n_55_n','n_50_n','n_45_n','n_40_n','n_35_n','n_30_n','n_25_n','n_20_n','n_15_n','n_10_n','n_05_n','n_00_n','s_00_s','s_05_s','s_10_s','s_15_s','s_20_s','s_25_s','s_30_s','s_35_s','s_40_s','s_45_s','s_50_s','s_55_s']

In [ ]:
for year_index in range(0,3):
    for i in range (0,18000):
        try:          
            year = 2000 + 10*year_index
            lat_i = int(i/600)-1
            latitude = lat_list[lat_i]
            path = 'users/1801213219/Globeland30_2000/Globeland30'+latitude+'_'+str(year)
            image_lc = ee.Image(path)
            arableland_pro= image_lc.gte(2)
            arableland = image_lc.updateMask(arableland_pro).lte(19).unmask(0)
            forest_pro= image_lc.gte(20)
            forest = image_lc.updateMask(forest_pro).lte(29).unmask(0)
            grassland_pro= image_lc.gte(30)
            grassland = image_lc.updateMask(grassland_pro).lte(39).unmask(0)
            shrubland_pro= image_lc.gte(40)
            shrubland = image_lc.updateMask(shrubland_pro).lte(49).unmask(0)
            wetlands_pro= image_lc.gte(50)
            wetlands = image_lc.updateMask(wetlands_pro).lte(59).unmask(0)
            waterbody_pro= image_lc.gte(60)
            waterbody = image_lc.updateMask(waterbody_pro).lte(69).unmask(0)
            tundra_pro= image_lc.gte(70)
            tundra = image_lc.updateMask(tundra_pro).lte(79).unmask(0)
            atificalsurface_pro= image_lc.gte(80)
            atificalsurface = image_lc.updateMask(atificalsurface_pro).lte(89).unmask(0)
            bareground_pro= image_lc.gte(90)
            bareground = image_lc.updateMask(bareground_pro).lte(99).unmask(0)
            permanentsnow_pro= image_lc.gte(100)
            permanentsnow = image_lc.updateMask(permanentsnow_pro).lte(120).unmask(0)
            Land_pro= image_lc.gte(2)
            Land = image_lc.updateMask(Land_pro).lte(120).unmask(0)
            cond = ee.Image.cat(
            arableland.eq(1).rename('arableland'),
            forest.eq(1).rename('forest'),
            grassland.eq(1).rename('grassland'),
            shrubland.eq(1).rename('shrubland'),
            wetlands.eq(1).rename('wetlands'),
            waterbody.eq(1).rename('waterbody'),
            tundra.eq(1).rename('tundra'),
            atificalsurface.eq(1).rename('atificalsurface'),
            bareground.eq(1).rename('bareground'),
            permanentsnow.eq(1).rename('permanentsnow'),
            Land.eq(1).rename('Land'))
            i_g = yrange-i-2
            y1 = ymin + i_g*dx
            y2 = ymin + (i_g+1)*dx
            grid = generateGrid(xmin, xmax, dx, y1, y2)
            npsRes = cond.reduceRegions(grid, ee.Reducer.sum(), 60)
            npsRes = npsRes.map(func_feh)
            name='Globeland30_'+'Globe_'+str(i)+'_Half2.csv'
            out_dir = 'G:\\LUCC_China\\Globeland30\\GEE_GlobeRatio\\Y'+str(year)+'\\Half2'
            out_ndvi_stats = os.path.join(out_dir, name)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
            url = npsRes.getDownloadURL(filetype="csv", selectors=['TARGET_FID','arableland',
                                                                    'forest',
                                                                    'grassland',
                                                                    'shrubland',
                                                                    'wetlands',
                                                                    'waterbody',
                                                                    'tundra',
                                                                    'atificalsurface',
                                                                    'bareground',
                                                                    'permanentsnow',
                                                                    'Land'], filename=name)
            print(url)
            print(out_ndvi_stats)
            get_request(url)
        except Exception as e:
            print(e)       